In [ ]:
# Import relevant Built-in Libraries & Functions
import cv2
from keras.models import load_model
from keras.utils.generic_utils import CustomObjectScope

In [ ]:
# Import Models
from models.unets import Unet2D
from models.FCN import FCN_Vgg16_16s
from models.SegNet import SegNet

In [ ]:
# Import custom Utilities
from utils.learning.metrics import dice_coef, precision, recall
from utils.BilinearUpSampling import BilinearUpSampling2D
from utils.io.data import load_data, save_results, save_rgb_results, save_history, load_test_images, DataGen

In [ ]:
# Define variables for Testing
input_dim_x = 224
input_dim_y = 224
color_space = 'rgb'
path = #'./data/Medetec_foot_ulcer_224/'
weight_file_name = #'2022-05-09 01%3A53%3A15.480800.hdf5'
pred_save_path = #'2022-05-09 01%3A53%3A15.480800/'

In [ ]:
# Data Generation for Testing
data_gen = DataGen(path, split_ratio=0.0, x=input_dim_x, y=input_dim_y, color_space=color_space)
x_test, test_label_filenames_list = load_test_images(path)

In [ ]:
# UNet Model Test
unet2d = Unet2D(n_filters=64, input_dim_x=input_dim_x, input_dim_y=input_dim_y, num_channels=3)\
model = load_model('./training_history/' + weight_file_name, 
                   custom_objects={'recall':recall,
                                 'precision':precision,
                                 'dice_coef': dice_coef,
                                 'relu6':relu6,
                                 'DepthwiseConv2D':DepthwiseConv2D,
                                 'BilinearUpsampling':BilinearUpsampling})

In [ ]:
# sep_unet = Separable_Unet2D(n_filters=64, input_dim_x=input_dim_x, input_dim_y=input_dim_y, num_channels=3)
# model, model_name = sep_unet.get_sep_unet_v2()
# model = load_model('./training_history/' + weight_file_name, 
#                    custom_objects={'dice_coef': dice_coef,
#                                  'relu6':relu6,
#                                  'DepthwiseConv2D':DepthwiseConv2D,
#                                  'BilinearUpsampling':BilinearUpsampling})

In [ ]:
# FCN Model Test
model, model_name = FCN_Vgg16_16s(input_shape=(input_dim_x, input_dim_y, 3))
with CustomObjectScope({'BilinearUpSampling2D':BilinearUpSampling2D}):
    model = load_model('./training_history/' + weight_file_name, custom_objects={'dice_coef': dice_coef})

In [ ]:
# Save Test Results
for image_batch, label_batch in data_gen.generate_data(batch_size=len(x_test), test=True):
    prediction = model.predict(image_batch, verbose=1)
    save_results(prediction, 'rgb', path + 'test/predictions/' + pred_save_path, test_label_filenames_list)
    break